# Diversity and Inclusion Company Score Card
![DnI Image](https://gresb.com/wp-content/uploads/2019/08/People_Graphic-768x398.jpg)

In [1]:
import eikon as ek
import plotly.offline as offline
from plotly import graph_objs as go
from IPython.display import display, Markdown
offline.offline.init_notebook_mode(connected=True)

In [2]:
ek.set_app_key('-- YOUR EIKON KEY HERE --')
symbol = 'MSFT.O'

## Company fundamental information

In [3]:
d1, err = ek.get_data(symbol, fields = ['TR.CompanyName', 'TR.TRBCIndustryGroup', 'TR.HeadquartersCountry', 'TR.TRBCIndustryGroupCode', 'TR.HQCountryCode'])
inst = {
    'RIC': symbol,
    'Name': d1.iloc[0][1],
    'IndustryGroup': d1.iloc[0][2],
    'Headquarters': d1.iloc[0][3],
    'IndustryCode': d1.iloc[0][4],
    'CountryCode': d1.iloc[0][5],
}

for key, val in inst.items():
    display(Markdown('**{}:** {}'.format(key, val)))

**RIC:** MSFT.O

**Name:** Microsoft Corp

**IndustryGroup:** Software & IT Services

**Headquarters:** United States of America

**IndustryCode:** 572010

**CountryCode:** US

## Use screener to get country and industry peers

In [4]:
# define the ranking function
rFields = ['TR.CommonName', 'TR.TRDIRScore']
def getRank(screenerQuery):
    iall, err = ek.get_data(instruments = screenerQuery, fields = rFields)
    iall_sorted = iall.sort_values('TRDIR Score', ascending=0).reset_index(drop=True)
    iRank = iall_sorted.loc[iall_sorted['Company Common Name'] == inst['Name']].index[0]
    sRank = str(iRank + 1) + "/" + str(iall_sorted.index.max() + 1)
    return sRank, iall_sorted

In [5]:
# Get overall rank
overallRank, overallList = getRank('SCREEN(U(IN(Equity(active,public,primary))), TR.TRDIRScore>=0, CURN=USD)')
display(Markdown('**Overall Rank:** {}'.format(overallRank)))

# Get country rank
countryRank, countryList = getRank('SCREEN(U(IN(Equity(active,public,primary))), TR.TRDIRScore>=0, IN(TR.HQCountryCode,' + inst['CountryCode'] + '), CURN=USD)')
display(Markdown('**Country Rank:** {}'.format(countryRank)))

# Get industry rank
industryRank, industryList = getRank('SCREEN(U(IN(Equity(active,public,primary))), TR.TRDIRScore>=0, IN(TR.TRBCIndustryGroupCode,' + str(inst['IndustryCode']) + '), CURN=USD)')
display(Markdown('**Industry Group Rank:** {}'.format(industryRank)))

**Overall Rank:** 447/2490

**Country Rank:** 80/571

**Industry Group Rank:** 13/116

## Top and Bottom 5 in this industry

In [6]:
industryList[['Company Common Name', 'TRDIR Score']].iloc[0: 5]

,Company Common Name,TRDIR Score
0,Accenture PLC,83.25
1,Fujitsu Ltd,69.00
2,SAP SE,67.75
3,NortonLifeLock Inc,67.50
4,Salesforce.Com Inc,67.00


In [7]:
industryList[['Company Common Name', 'TRDIR Score']].iloc[-5: industryList.shape[0]]

,Company Common Name,TRDIR Score
111,Paylocity Holding Corp,37.50
112,Alarm.com Holdings Inc,37.50
113,Ebix Inc,36.75
114,Nemetschek SE,36.00
115,Colopl Inc,33.00


## Comparison with Country and Industry benchmarks

In [8]:
aFields = ['TR.CommonName', 'TR.TRDIRScore', 'TR.TRDIRDiversityScore', 'TR.TRDIRPeopleDevelopmentScore', 'TR.TRDIRInclusionScore', 'TR.TRDIRControvScore']
dnIdata, err = ek.get_data(instruments = [symbol, countryList.iloc[0]['Instrument'], industryList.iloc[0]['Instrument']], fields = aFields)
compare = dnIdata.transpose()
compare.columns = ['', 'Top Ranked {}'.format(inst['CountryCode']), inst['IndustryGroup']]
compare

,,Top Ranked US,Software & IT Services
Instrument,MSFT.O,BLK.N,ACN.N
Company Common Name,Microsoft Corp,BlackRock Inc,Accenture PLC
TRDIR Score,60.5,78.25,83.25
TRDIR Diversity Score,54,44,69
TRDIR People Development Score,50,73,76
TRDIR Inclusion Score,50,96,88
TRDIR Controversies Score,88,100,100


In [9]:
# Plot it in the bar chart
def generateBar(index):
    cName = compare.columns[index]
    return go.Bar(x=compare.index[2:], y=compare[cName][2:], name=compare[cName]['Company Common Name'])

fig = go.Figure([generateBar(0), generateBar(1), generateBar(2)])
fig.update_layout(title='Best in Class Score Comparison')
fig.show()